# Load dependencies

In [ ]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from scipy.misc import imresize

from flip_gradient import flip_gradient
from utils import *
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Read data into memory

In [ ]:
target_names = ['W','N1','N2','N3','R']

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
num_subjects_physionet = 20
num_subjects_hospital = 17
#data_physionet = pickle.load(open('C:/Users/ramuol/Documents/dataOut.pkl','rb'))
#data_hospital = pickle.load(open('C:/Users/ramuol/Documents/hDataOut.pkl', 'rb'))
data_physionet = pickle.load(open('/home/jaskmo/Documents/DataCollection/sleep-edfx/PickleJar/dataOut.pkl','rb'))
data_hospital = pickle.load(open('/home/jaskmo/Documents/DataCollection/sleep-edfx/PickleJar/hDataOut.pkl','rb'))

#Get random index for training and test for the physionet data (3 subjects are kept for test)
random_perm_physionet = np.random.permutation(num_subjects_physionet)
idx_tmp_physionet = random_perm_physionet[range(num_subjects_physionet - 3)]
idx_test_physionet = random_perm_physionet[(num_subjects_physionet - 3) : num_subjects_physionet]

#Get random index for training and test for the hospital data
random_perm_hospital = np.random.permutation(num_subjects_hospital)
idx_tmp_hospital = random_perm_hospital[range(num_subjects_hospital - 3)]
idx_test_hospital = random_perm_hospital[(num_subjects_hospital- 3) : num_subjects_hospital]
        
#Get the datasets for the data from physionet
inputs_train_phys, targets_train_phys, inputs_val_phys, targets_val_phys, inputs_test_phys, targets_test_phys = get_data_complete(
   idx_tmp_physionet, idx_test_physionet, data_physionet, 'physionet')
print("Physionet data in memory")

inputs_train_hosp, targets_train_hosp, inputs_val_hosp, targets_val_hosp, inputs_test_hosp, targets_test_hosp = get_data_complete(
   idx_tmp_hospital, idx_test_hospital, data_hospital, 'hospital')
print("Hospital data in memory")

# Create a mixed dataset for TSNE visualization
num_test = 25
combined_test_imgs = np.vstack([inputs_train_phys[:num_test], inputs_train_hosp[:num_test]])
combined_test_labels = np.vstack([targets_train_phys[:num_test], targets_train_hosp[:num_test]])
combined_test_domain = np.vstack([np.tile([1., 0.], [num_test, 1]),
       np.tile([0., 1.], [num_test, 1])])

## Build model

In [ ]:
batch_size = 30
beta = 0.001

class MNISTModel():
    """Simple MNIST domain adaptation model."""
    def __init__(self):
        self.parameters = []
        self._build_model()
    
    def _build_model(self):
        
        self.noClasses = 5
        self.X = tf.placeholder(tf.uint8, [None, 224, 224, 3])
        self.y = tf.placeholder(tf.float32, [None, self.noClasses])
        self.domain = tf.placeholder(tf.float32, [None, 2])
        self.l = tf.placeholder(tf.float32, [])
        self.train = tf.placeholder(tf.bool, [])
        
        X_input = tf.cast(self.X, tf.float32)/255
               
        # CNN model for feature extraction
        with tf.variable_scope('feature_extractor'):
            
            # First CNN
            W_conv0 = weight_variable([3, 3, 3, 64])
            b_conv0 = bias_variable([64])
            h_conv0 = tf.nn.relu(conv2d(X_input, W_conv0) + b_conv0)
           
            W_conv1 = weight_variable([3, 3, 64, 64])
            b_conv1 = bias_variable([64])
            h_conv1 = tf.nn.relu(conv2d(h_conv0, W_conv1) + b_conv1)
            h_pool1 = max_pool_2x2(h_conv1)
            
            # Second CNN
            W_conv2 = weight_variable([3, 3, 64, 128])
            b_conv2 = bias_variable([128])
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
                       
            W_conv3 = weight_variable([3, 3, 128, 128])
            b_conv3 = bias_variable([128])
            h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3) + b_conv3)
            h_pool3 = max_pool_2x2(h_conv3)
                                    
            # Third CNN
            W_conv4 = weight_variable([3, 3, 128, 256])
            b_conv4 = bias_variable([256])
            h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
                        
            W_conv5 = weight_variable([3, 3, 256, 256])
            b_conv5 = bias_variable([256])
            h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5) + b_conv5)
                             
            W_conv6 = weight_variable([3, 3, 256, 256])
            b_conv6 = bias_variable([256])
            h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)
            h_pool6 = max_pool_2x2(h_conv6)
            
            # Fourth CNN
            W_conv7 = weight_variable([3, 3, 256, 512])
            b_conv7 = bias_variable([512])
            h_conv7 = tf.nn.relu(conv2d(h_pool6, W_conv7) + b_conv7)
            
            W_conv8 = weight_variable([3, 3, 512, 512])
            b_conv8 = bias_variable([512])
            h_conv8 = tf.nn.relu(conv2d(h_conv7, W_conv8) + b_conv8)
                            
            W_conv9 = weight_variable([3, 3, 512, 512])
            b_conv9 = bias_variable([512])
            h_conv9 = tf.nn.relu(conv2d(h_conv8, W_conv9) + b_conv9)
            h_pool9 = max_pool_2x2(h_conv9)
                        
            # Fifth CNN
            W_conv10 = weight_variable([3, 3, 512, 512])
            b_conv10 = bias_variable([512])
            h_conv10 = tf.nn.relu(conv2d(h_pool9, W_conv10) + b_conv10)
                        
            W_conv11 = weight_variable([3, 3, 512, 512])
            b_conv11 = bias_variable([512])
            h_conv11 = tf.nn.relu(conv2d(h_conv10, W_conv11) + b_conv11)
                            
            W_conv12 = weight_variable([3, 3, 512, 512])
            b_conv12 = bias_variable([512])
            h_conv12 = tf.nn.relu(conv2d(h_conv11, W_conv12) + b_conv12)
            h_pool12 = max_pool_2x2(h_conv12)
                        
            # The domain-invariant feature
            self.shape = int(np.prod(h_pool12.get_shape()[1:]))

            self.feature = tf.reshape(h_pool12, [-1, self.shape])
            
        # MLP for class prediction
        with tf.variable_scope('label_predictor'):
            
            # Switches to route target examples (second half of batch) differently
            # depending on train or test mode.
            all_features = lambda: self.feature
            source_features = lambda: tf.slice(self.feature, [0, 0], [(int)(batch_size / 2), -1])
            classify_feats = tf.cond(self.train, source_features, all_features)
            
            all_labels = lambda: self.y
            source_labels = lambda: tf.slice(self.y, [0, 0], [int(batch_size / 2), -1])
            self.classify_labels = tf.cond(self.train, source_labels, all_labels)
            
            W_fc0 = dense_variable([self.shape, 2048])
            b_fc0 = bias_variable([2048])
            h_fc0 = tf.layers.dropout(tf.nn.relu(tf.matmul(classify_feats, W_fc0) + b_fc0), rate=0.5)

            W_fc1 = dense_variable([2048, 1024])
            b_fc1 = bias_variable([1024])
            h_fc1 = tf.layers.dropout(tf.nn.relu(tf.matmul(h_fc0, W_fc1) + b_fc1), rate=0.5)

            W_fc2 = dense_variable([1024, self.noClasses])
            b_fc2 = bias_variable([self.noClasses])
            logits = tf.matmul(h_fc1, W_fc2) + b_fc2
            
            self.pred = tf.nn.softmax(logits)
            self.pred_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.classify_labels)

        # Small MLP for domain prediction with adversarial loss
        with tf.variable_scope('domain_predictor'):
            
            # Flip the gradient when backpropagating through this operation
            feat = flip_gradient(self.feature, self.l)
            d_W_fc0 = dense_variable([self.shape, 2048])
            d_b_fc0 = bias_variable([2048])
            d_h_fc0 = tf.layers.dropout(tf.nn.relu(tf.matmul(feat, d_W_fc0) + d_b_fc0), 0.5)
            
            d_W_fc1 = dense_variable([2048, 1024])
            d_b_fc1 = bias_variable([1024])
            d_h_fc1 = tf.layers.dropout(tf.nn.relu(tf.matmul(d_h_fc0, d_W_fc1) + d_b_fc1), rate=0.5)
            
            d_W_fc2 = dense_variable([1024, 2])
            d_b_fc2 = bias_variable([2])
            d_logits = tf.matmul(d_h_fc1, d_W_fc2) + d_b_fc2
            
            self.domain_pred = tf.nn.softmax(d_logits)
            self.domain_loss = tf.nn.softmax_cross_entropy_with_logits(logits=d_logits, labels=self.domain)

## Function initializing the weights

In [ ]:
def initialize_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
    vgg_vars = not_initialized_vars[:26]
    other_vars = not_initialized_vars[26:]
    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(other_vars))
     
    weight_file= '/home/jaskmo/Documents/DataCollection/sleep-edfx/PickleJar/vgg16_weights.npz'
#     weight_file= 'D:/vgg16_weights.npz'

    weights = np.load(weight_file)
    keys = sorted(weights.keys())
    for i, k in enumerate(keys):
        if i < 26:
            sess.run(vgg_vars[i].assign(weights[k]))

## Losses and accuracies

In [ ]:
# Build the model graph
graph = tf.get_default_graph()
with graph.as_default():
    model = MNISTModel()
    
    learning_rate = tf.placeholder(tf.float32, [])
    
    pred_loss = tf.reduce_mean(model.pred_loss)
    domain_loss = tf.reduce_mean(model.domain_loss)
    total_loss = pred_loss + domain_loss
    regularize = tf.contrib.layers.l2_regularizer(beta)
    params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    reg_term = sum([regularize(param) for param in params])
    total_loss += reg_term

    regular_train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(pred_loss)
    dann_train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(total_loss)
    
    # Evaluation
    correct_label_pred = tf.equal(tf.argmax(model.classify_labels, 1), tf.argmax(model.pred, 1))
    label_acc = tf.reduce_mean(tf.cast(correct_label_pred, tf.float32))
    correct_domain_pred = tf.equal(tf.argmax(model.domain, 1), tf.argmax(model.domain_pred, 1))
    domain_acc = tf.reduce_mean(tf.cast(correct_domain_pred, tf.float32))

## Training and evaluation

In [ ]:
noEpochs = 50

def train_and_evaluate(training_mode, graph, model, verbose=1):
    """Helper to run the model with different training modes."""
    _domain_loss_train, _domain_acc_train, _label_loss_train, _label_acc_train, _total_loss_train = [], [], [], [], []
    
    domain_loss_train, domain_loss_test, domain_loss_valid = [], [], []
    domain_acc_train, domain_acc_test, domain_acc_valid = [], [], []
    label_loss_train, label_loss_test, label_loss_valid = [], [], []
    label_acc_train, label_acc_test, label_acc_test_phys, label_acc_test_hosp, label_acc_valid = [], [], [], [], []
    total_loss_train, total_loss_valid = [], []
    
    domain_labels = np.vstack([np.tile([1, 0], [batch_size // 2, 1]),
                                np.tile([0, 1], [batch_size // 2, 1])]) 
    
    with tf.Session(graph=graph, config=config) as sess:
        initialize_uninitialized(sess)
        
        if training_mode == 'dann':
            saverDanet = tf.train.Saver()
            trainFile = open('trainPerformanceDann.txt', 'w')
            validFile = open('validPerformanceDann.txt', 'w')
            testFile = open('testPerformanceDann.txt', 'w')
        elif training_mode == 'source':
            saverSource = tf.train.Saver()
            trainFile = open('trainPerformanceSource.txt', 'w')
            validFile = open('validPerformanceSource.txt', 'w')
            testFile = open('testPerformanceSource.txt', 'w')
        else:
            saverTarget = tf.train.Saver()
            trainFile = open('trainPerformanceTarget.txt', 'w')
            validFile = open('validPerformanceTarget.txt', 'w')
            testFile = open('testPerformanceTarget.txt', 'w')
        
        # Training loop
        for ii in range(noEpochs):
            print("Epoch number: ", ii+1)
            inputs_tr_phys,targets_tr_phys = get_data_training_epoch(inputs_train_phys,targets_train_phys)
            inputs_tr_hosp, targets_tr_hosp = get_data_training_epoch(inputs_train_hosp, targets_train_hosp)
  
            # Adaptation param and learning rate schedule as described in the paper
            p = float(ii) / noEpochs
            l = 2. / (1. + np.exp(-10. * p)) - 1
#             lr = 0.01 / ((1. + 10 * p)**0.75)
#             lr = 0.0001
            if ii < 20:
                lr = 0.0001
            elif ii >= 20 and ii < 40:
                lr = 0.00005
            else:
                lr = 0.00001
                    
            # Training step
            thisNo = 0
            if training_mode == 'dann':
                ################## TRAINING ########################
                for batch in iterate_minibatches_dann(inputs_tr_phys, targets_tr_phys, inputs_tr_hosp, 
                                                      targets_tr_hosp, batch_size, shuffle=True):               
                    X, y = batch #Get the values 
                    if len(y) == batch_size:

                        thisNo += 1
                        _, batch_loss, dloss, ploss, d_acc, p_acc = \
                            sess.run([dann_train_op, total_loss, domain_loss, pred_loss, domain_acc, label_acc],
                                     feed_dict={model.X: X, model.y: y, model.domain: domain_labels,
                                                model.train: True, model.l: l, learning_rate: lr})
                        _domain_loss_train.append(dloss)
                        _domain_acc_train.append(d_acc)
                        _label_loss_train.append(ploss)
                        _label_acc_train.append(p_acc)
                        _total_loss_train.append(batch_loss)
                        
                # Append for each epoch
                domain_loss_train.append(np.mean(_domain_loss_train))
                domain_acc_train.append(np.mean(_domain_acc_train))
                label_loss_train.append(np.mean(_label_loss_train))
                label_acc_train.append(np.mean(_label_acc_train))
                total_loss_train.append(np.mean(_total_loss_train))
                print('TRAINING:   Total loss: %f, Domain acc: %f, Label acc: %f, Label loss: %f, Domain adapt para: %f  lr: %f' % \
                      (np.mean(_total_loss_train), np.mean(_domain_acc_train), np.mean(_label_acc_train), np.mean(_label_loss_train), l, lr))

                trainFile.write("%s\n" % [np.mean(_total_loss_train), np.mean(_domain_acc_train), np.mean(_label_acc_train), np.mean(_label_loss_train), l, lr])
                _domain_loss_train, _domain_acc_train, _label_loss_train, _label_acc_train, _total_loss_train = [], [], [], [], []
                ################## TRAINING ENDED ##########################
                
                ################## VALIDATION ########################
                _domain_loss_valid, _domain_acc_valid, _label_loss_valid, _label_acc_valid, _total_loss_valid = [], [], [], [], []
                batchNo = 0
                for batch in iterate_minibatches_dann(inputs_val_phys, targets_val_phys, inputs_val_hosp,
                                          targets_val_hosp, batch_size, shuffle=False):                
                    X, y = batch  # Get the values
                    if len(y) == batch_size:
                        batchNo += 1
                        # Feed_dict and fetches
                        feed_dict = {model.X: X, model.y: y, model.domain: domain_labels, model.train: True}
                        fetches = [total_loss, domain_loss, pred_loss, domain_acc, label_acc]
                        batch_loss, dloss, ploss, d_acc, p_acc = sess.run(fetches, feed_dict=feed_dict)
                        # Get the loss for a single batch averaged

                        _domain_loss_valid.append(dloss)
                        _domain_acc_valid.append(d_acc)
                        _label_loss_valid.append(ploss)
                        _label_acc_valid.append(p_acc)
                        _total_loss_valid.append(batch_loss)
                # Append loss and accuracy
                label_loss_valid.append(np.mean(_label_loss_valid))
                label_acc_valid.append(np.mean(_label_acc_valid))
                domain_loss_valid.append(np.mean(_domain_loss_valid))
                domain_acc_valid.append(np.mean(_domain_acc_valid))
                total_loss_valid.append(np.mean(_total_loss_valid))
                ################## VALIDATION ENDED ########################
                print('VALIDATION: Total loss: %f, Domain acc: %f, Label acc: %f, Label loss: %f ' % \
                  (np.mean(_total_loss_valid), np.mean(_domain_acc_valid), np.mean(_label_acc_valid), np.mean(_label_loss_valid)))
            
                validFile.write("%s\n" % [np.mean(_total_loss_valid), np.mean(_domain_acc_valid), np.mean(_label_acc_valid), np.mean(_label_loss_valid)])        
                ################## TESTING ########################
                # IS FURHTER BELOW                  
            elif training_mode == 'source':
                batchNo = 0 
                ################## TRAINING ########################
                for batch in iterate_minibatches(inputs_tr_phys, targets_tr_phys, batch_size, shuffle=True):
                    X, y = batch
                    if len(y) == batch_size:
                        batchNo += 1
                        feed_dict = {model.X: X, model.y: y, model.train: False, model.l: l, learning_rate: lr}
                        fetches = [regular_train_op, pred_loss, label_acc]
                        _, ploss, p_acc = sess.run(fetches, feed_dict=feed_dict)
                        _label_loss_train.append(ploss)
                        _label_acc_train.append(p_acc)
                        
                # Append for each epoch
                label_loss_train.append(np.mean(_label_loss_train))
                label_acc_train.append(np.mean(_label_acc_train))
                print('TRAINING:   Label acc: %f, Label loss: %f, lr: %f' % \
                      (np.mean(_label_acc_train), np.mean(_label_loss_train), lr))

                trainFile.write("%s\n" % [np.mean(_label_acc_train), np.mean(_label_loss_train), lr])
                _label_loss_train, _label_acc_train = [], []
                ################## TRAINING ENDED ########################
                ################## VALIDATION ########################
                _label_loss_valid, _label_acc_valid = [], []
                batchNo = 0
                for batch in iterate_minibatches(inputs_val_phys, targets_val_phys, batch_size, shuffle=False):                
                    X, y = batch  # Get the values
                    if len(y) == batch_size:
                        batchNo += 1
                        # Feed_dict and fetches
                        feed_dict = {model.X: X, model.y: y, model.train: False}
                        fetches = [pred_loss, label_acc]
                        ploss, p_acc = sess.run(fetches, feed_dict=feed_dict)
                        # Get the loss for a single batch averaged
                        _label_loss_valid.append(ploss)
                        _label_acc_valid.append(p_acc)
                        
                # Append loss and accuracy
                label_loss_valid.append(np.mean(_label_loss_valid))
                label_acc_valid.append(np.mean(_label_acc_valid))
                print('VALIDATION: Label acc: %f, Label loss: %f ' % (np.mean(_label_acc_valid), np.mean(_label_loss_valid)))
            
                validFile.write("%s\n" % [np.mean(_label_acc_valid), np.mean(_label_loss_valid)])    
                ################## VALIDATION ENDED ########################
    
                ################## TESTING ########################
                # IS FURHTER BELOW      
            elif training_mode == 'target':
                ################## TRAINING ########################
                batchNo = 0
                for batch in iterate_minibatches(inputs_tr_hosp, targets_tr_hosp, batch_size, shuffle=True):
                    X, y = batch
                    if len(y) == batch_size:
                        feed_dict = {model.X: X, model.y: y, model.train: False, model.l: l, learning_rate: lr}
                        fetches = [regular_train_op, pred_loss, label_acc]
                        _, ploss, p_acc = sess.run(fetches, feed_dict=feed_dict)
                        _label_loss_train.append(ploss)
                        _label_acc_train.append(p_acc)
                        
                # Append for each epoch
                label_loss_train.append(np.mean(_label_loss_train))
                label_acc_train.append(np.mean(_label_acc_train))
                print('TRAINING:   Label acc: %f, Label loss: %f, lr: %f' % \
                      (np.mean(_label_acc_train), np.mean(_label_loss_train), lr))

                trainFile.write("%s\n" % [np.mean(_label_acc_train), np.mean(_label_loss_train), lr])
                _label_loss_train, _label_acc_train = [], []
                ################## TRAINING ENDED ########################
                
                ################## VALIDATION ########################
                _label_loss_valid, _label_acc_valid = [], []
                batchNo = 0
                for batch in iterate_minibatches(inputs_val_hosp, targets_val_hosp, batch_size, shuffle=False):                
                    X, y = batch  # Get the values
                    if len(y) == batch_size:
                        batchNo += 1
                        # Feed_dict and fetches
                        feed_dict = {model.X: X, model.y: y, model.train: False}
                        fetches = [pred_loss, label_acc]
                        ploss, p_acc = sess.run(fetches, feed_dict=feed_dict)
                        # Get the loss for a single batch averaged
                        _label_loss_valid.append(ploss)
                        _label_acc_valid.append(p_acc)
                        
                # Append loss and accuracy
                label_loss_valid.append(np.mean(_label_loss_valid))
                label_acc_valid.append(np.mean(_label_acc_valid))
                print('VALIDATION: Label acc: %f, Label loss: %f ' % (np.mean(_label_acc_valid), np.mean(_label_loss_valid)))
            
                validFile.write("%s\n" % [np.mean(_label_acc_valid), np.mean(_label_loss_valid)])    
                ################## VALIDATION ENDED ########################
               
                ################## TESTING ON SOURCE DATA ########################
                _temp_pred = []
                _temp_target = []
                _label_acc_test = []
                batchNo = 0
                for batch in iterate_minibatches(inputs_test_hosp, targets_test_hosp, batch_size, shuffle=False):
                    X, y = batch  # Get the values
                    if len(y) == batch_size:
                        batchNo += 1

                        # Feed_dict and fetches
                        feed_dict = {model.X: X, model.y: y, model.train: False}
                        fetches = [label_acc, model.pred]
                        p_acc, prediction = sess.run(fetches, feed_dict=feed_dict)

                        _temp_pred = np.append(_temp_pred, np.argmax(prediction, axis=1))

                        # Convert targets from onehot
                        _temp_target = np.append(_temp_target, [np.where(r == 1)[0][0] for r in y])
                        _label_acc_test.append(p_acc)

                # Confusion matrix
                conf_mat = confusion_matrix(_temp_target, _temp_pred)
                print(conf_mat)

                # Per class metrics
                class_report = classification_report(_temp_target, _temp_pred, target_names=target_names)
                print(class_report)

                print(np.mean(_label_acc_test))

                testFile.write("%s\n" % [conf_mat])
                testFile.write("%s\n" % [class_report])
                testFile.write("%s\n" % [np.mean(_label_acc_test)])
            
            if training_mode == 'dann' or training_mode == 'source':
                _temp_pred = []
                _temp_target = []
                _label_acc_test = []
                batchNo = 0
                for batch in iterate_minibatches(inputs_test_hosp, targets_test_hosp, batch_size, shuffle=False):
                    X, y = batch  # Get the values
                    if len(y) == batch_size:
                        batchNo += 1

                        # Feed_dict and fetches
                        feed_dict = {model.X: X, model.y: y, model.train: False}
                        fetches = [label_acc, model.pred]
                        p_acc, prediction = sess.run(fetches, feed_dict=feed_dict)

                        _temp_pred = np.append(_temp_pred, np.argmax(prediction, axis=1))

                        # Convert targets from onehot
                        _temp_target = np.append(_temp_target, [np.where(r == 1)[0][0] for r in y])
                        _label_acc_test.append(p_acc)

                # Confusion matrix
                conf_mat = confusion_matrix(_temp_target, _temp_pred)
                print(conf_mat)

                # Per class metrics
                class_report = classification_report(_temp_target, _temp_pred, target_names=target_names)
                print(class_report)

                print(np.mean(_label_acc_test))

                testFile.write("%s\n" % [conf_mat])
                testFile.write("%s\n" % [class_report])
                testFile.write("%s\n" % [np.mean(_label_acc_test)])
        
        # Get the outputs
        test_emb = sess.run(model.feature, feed_dict={model.X: combined_test_imgs})
        testFile.close()
        validFile.close()
        testFile.close()
        
        
        if training_mode == 'target':
            saverTarget.save(sess, "/home/jaskmo/Documents/programering/02456-deep-learning/Project/RES/model_target.ckpt")
        elif training_mode == 'source':
            saverSource.save(sess, "/home/jaskmo/Documents/programering/02456-deep-learning/Project/RES/model_source.ckpt")
        else:
            saverDanet.save(sess, "/home/jaskmo/Documents/programering/02456-deep-learning/Project/RES/model_danet.ckpt")
                
        return test_emb   


#print('\nSource only training')
#source_emb = train_and_evaluate('source', graph, model)

#print('\nTarget only training')
#target_emb = train_and_evaluate('target', graph, model)

print('\nDomain adaptation training')
dann_emb = train_and_evaluate('dann', graph, model)